# Kaggle TPS 2021 Octboer

## Optuna LightGBM Starter

### Check GPU

In [2]:
# Google Colab
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Oct  4 04:20:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    41W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Check Memory

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


### PIP Installs

Uninstall the colab default lightgbm, since it is not set up for GPU

In [5]:
!pip uninstall lightgbm
!pip install lightgbm --install-option=--gpu

Found existing installation: lightgbm 2.2.3
Uninstalling lightgbm-2.2.3:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/lightgbm-2.2.3.dist-info/*
    /usr/local/lib/python3.7/dist-packages/lightgbm/*
Proceed (y/n)? y
  Successfully uninstalled lightgbm-2.2.3
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:232: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
     |████████████████████████████████| 1.5 MB 5.0 MB/s 
Skipping wheel build for lightgbm, due to binaries being disabled for it.
    Running setup.py install for lightgbm ... done


n
For now, git clone the lightgbm from the repo.  (There is possibly another GPU friendly install option, but it hasn't been verified to work)

Note that this will take up space, so this is being done before mounting Google Drive



In [ ]:
# This works on one notebook but not in this latest copy
#!git clone --recursive https://github.com/Microsoft/LightGBM
#! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;    
# unfortunately I forgot where I got the above, so cannot give them adequate credit

fatal: destination path 'LightGBM' already exists and is not an empty directory.


Other untried option

```!pip install lightgbm --install-option=--gpu```


In [7]:
# Next two lines are for Google Colab GPU and working with AutoGluon
!pip uninstall -y mkl
!pip install --upgrade mxnet-cu100

!pip install optuna

Found existing installation: mkl 2019.0
Uninstalling mkl-2019.0:
  Successfully uninstalled mkl-2019.0
     |████████████████████████████████| 352.6 MB 4.5 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 302 kB 5.0 MB/s 
     |████████████████████████████████| 80 kB 11.3 MB/s 
     |████████████████████████████████| 208 kB 81.3 MB/s 
     |████████████████████████████████| 75 kB 5.6 MB/s 
     |████████████████████████████████| 111 kB 80.0 MB/s 
     |████████████████████████████████| 144 kB 78.3 MB/s 
     |████████████████████████████████| 49 kB 7.5 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=5adf2ce58529c399ae7d49a6b2bdbd3933fd7ff483d8d2550203402467bd032c
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully

## Imports

In [8]:
import random
import os
from google.colab import output
import optuna
from pathlib import Path


from lightgbm import LGBMClassifier
import lightgbm as lgb
import json

import pandas as pd

from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
import pickle
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import roc_auc_score

#importing all the plot functions
from optuna.visualization import plot_edf
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

import logging

## Google Drive Mount

* This is used to save models and output
* Note that for this Kaggle competition, the train.csv and test.csv are quite large and take a while to upload and store.

In [9]:
working_dir = 'MyDrive/kaggle/202110'
drive_dir = '/content/drive/'
model_dir = 'models'
output_dir = 'output'
data_dir = 'data'
from google.colab import drive
drive.mount(drive_dir, force_remount=True)  # Do this once only

mounted_drive = f'{drive_dir}/{working_dir}' #''
os.chdir(mounted_drive)


Mounted at /content/drive/


In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/.kaggle"

If you want to directly download the kaggle files to your Google Colab environment

From https://www.analyticsvidhya.com/blog/2021/05/10-colab-tips-and-hacks-for-efficient-use-of-it/

https://github.com/Kaggle/kaggle-api

You can pull the datasets directly from Kaggle
1.  Go to your account page on kaggle.com and scroll to API to get your token
2.  Download the token 
3.  Copy it to Google Drive - most people put it in the .kaggle directory
4.  Set enviornment variable KAGGLE_CONFIG_DIR
5.  ```
! pip install kaggle
kaggle competitions download -c tabular-playground-series-sep-2021```
6.  You may need to add logic to move the files where you want them

In [ ]:
#!pip install kaggle
#!kaggle competitions download -c tabular-playground-series-sep-2021

In [ ]:
pd.set_option("display.max_rows", 999)
pd.set_option("display.max_columns", 999)

# Read csv files
#df = pd.read_csv('./train.csv')
#test_df=pd.read_csv('./test.csv')


## Logging

In [ ]:
# Right now this is not working as expected, so this is pending
#logging.basicConfig(filename='./artifacts/Kaggle_202109_lgbm.log',level=logging.INFO)

In [12]:
X = pd.read_csv(f'./{data_dir}/reduced_mem_imputed.csv')
test_X = pd.read_csv(f'./{data_dir}/reduced_mem_imputed_test.csv') # this is the Kaggle test dataset

In [13]:
y = X[['target']]
X = X.drop(columns = ['id', 'target'])

test_X = test_X.drop(columns = ['id'])

N_SPLITS = 5 # if you're using cross_score_validation
EARLY_STOPPING_ROUNDS = 300
SEED = 42

#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

Optuna 
* study - this is the tuning done over several iterations of different hyperparameters
* trial - this is the individual evaluation/trial of selected hyperparameters
* objective function - defines the logic that Optuna will use for a single trial
* callback - allows actions after the objective function has returned the ranking metric to Optuna.  

At this point Optuna does not save the best model for use after the study.  Instead it will return the best parameters of the study.  However the study can take a long time, so a callback is used to save the best model (as of yet) to a file.

**Note that there should be a sample file called best_{model_type} which will be replaced by the best model of the study.  Right now the job will err out, if the model isn't there. ** (And yes, more code can be written to automatically create that file or check it's existence, but it hasn't been done yet)

Due to the large training and test datasets, the objective function has been modified to work with train_test_split

In [ ]:
def lgbm_objective(trial):
   
    # hyperparameter setting, trial.suggest_uniform will suggest uniform hyperparameter
    
    
    kfolds = StratifiedKFold(n_splits=N_SPLITS, shuffle=True)

    # "lambda_l1": trial.suggest_float("lambda_l1", 1e-7, 10.0, log=True),
    #  "lambda_l2": trial.suggest_float("lambda_l2", 1e-7, 10.0, log=True),
    # "num_leaves": trial.suggest_int("num_leaves", 2, 256), - had error message due to n
    # "num_iterations": trial.suggest_int('num_iterations', 1000, 15000), - this was ok, 
    
    params = {
        "metric": "AUC",
        "verbosity": 0,
        "device": "gpu",
        "boosting_type": trial.suggest_categorical('boosting_type', ['gbdt']), # , 'dart'
        "subsample": trial.suggest_uniform('subsample', 0.5, 0.6),
        "learning_rate": trial.suggest_float('eta', 0.005, 0.015),
        "n_estimators": trial.suggest_int('n_estimators', 1000, 10000),
        "subsample_freq": 1,
        "reg_alpha": trial.suggest_int('reg_alpha', 10, 20),
        "reg_lambda": trial.suggest_int('reg_lambda', 10, 20),
        "max_depth": trial.suggest_int("max_depth", 5, 10),
        "num_leaves": trial.suggest_int("num_leaves", 2, 12),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),        
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "objective": "binary"
    }
    
    print(f'STARTING Params: {params}')
    lgbm_model = LGBMClassifier(**params,random_state=42)
    
    d_train = lgb.Dataset(X, label=y)
    # early_stopping_rounds=EARLY_STOPPING_ROUNDS,
    #https://www.kaggle.com/aharless/trying-to-do-k-fold-cv-on-lightgbm
    cv_results = lgb.cv(params, d_train, early_stopping_rounds=EARLY_STOPPING_ROUNDS, nfold=N_SPLITS, metrics = 'auc', seed=42)
    
    #print(cv_results)
    #results['auc-mean'][-1]
    auc = cv_results['auc-mean']
    print('Trial LGBM AUC',np.amax(auc), np.amin(auc), np.mean(auc))
    result = {}
    result['optuna_score'] = float(auc_score)
    
      
    result['scores'] = [float(np.amax(auc)), float(np.amin(auc)), float(np.mean(auc))]
    result['params'] = dict(keep_model.get_all_params())
    print(keep_model.get_all_params())

    with open('./artifacts/catboost_results.json', 'a') as f:
        json.dump(dict(result), f)
        f.write('\n')

    filename = f'./models/lgbm_{trial.number}'
    
    pickle.dump(lgbm_model, open(filename, 'wb'))
    
    return auc[-1] 


def lgbm_callback(study, trial):
  # from os import remove
    filename = f'./models/lgbm_{trial.number}'
    
    best_model_name = './models/best_lgbm'
    if study.best_trial.number == trial.number:
        os.remove(best_model_name)
        os.rename(filename, best_model_name)
    else:
        os.remove(filename)
    print(f'filename {filename} best_model_name{best_model_name} ; best_trial_number {study.best_trial.number}')

In [ ]:
kfolds = StratifiedKFold(n_splits=N_SPLITS, shuffle=True)
for train_index, test_index in kfolds.split(X_copy, y):
      print(train_index)
      X_train, X_test = X_copy.iloc[train_index], X_copy.iloc[test_index]
      #y_train, y_test = y[train_index], y[test_index]
      #eval_dataset = lgb.Dataset(X_test, label=y_test)

[     0      1      2 ... 999996 999997 999999]
[     0      1      3 ... 999997 999998 999999]
[     1      2      6 ... 999996 999998 999999]
[     0      1      2 ... 999996 999997 999998]
[     0      2      3 ... 999997 999998 999999]


In [ ]:
def kfold_objective(trial):
   
    
    # model training and evaluation
  

    
    kfolds = StratifiedKFold(n_splits=N_SPLITS, shuffle=True)

    # "lambda_l1": trial.suggest_float("lambda_l1", 1e-7, 10.0, log=True),
    #  "lambda_l2": trial.suggest_float("lambda_l2", 1e-7, 10.0, log=True),
    # "num_leaves": trial.suggest_int("num_leaves", 2, 256), - had error message due to n
    # "num_iterations": trial.suggest_int('num_iterations', 1000, 15000), - this was ok, 
    
    params = {
        "metric": "AUC",
        "verbosity": 0,
        "device": "gpu",
        "boosting_type": trial.suggest_categorical('boosting_type', ['gbdt']), # , 'dart'
        "subsample": trial.suggest_uniform('subsample', 0.5, 0.6),
        "learning_rate": trial.suggest_float('eta', 0.005, 0.015),
        "n_estimators": trial.suggest_int('n_estimators', 1000, 10000),
        "subsample_freq": 1,
        "reg_alpha": trial.suggest_int('reg_alpha', 10, 20),
        "reg_lambda": trial.suggest_int('reg_lambda', 10, 20),
        "max_depth": trial.suggest_int("max_depth", 5, 10),
        "num_leaves": trial.suggest_int("num_leaves", 2, 12),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),        
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "objective": "binary"
    }
    
    print(f'STARTING Params: {params}')
    model = LGBMClassifier(**params, random_state=42)
    
    
    
    print(f'Param: {params}')  # Printing this before the cross val runs, so we have an idea if this is a long running job
    auc_score = 0
    keep_model = LGBMClassifier(**params, random_state=42)
    auc = []
    #########################
    # Cross validation folds
    # We're not using lightgbm cv so we can try to save the best scoring model
    #########################
    for train_index, test_index in kfolds.split(X_copy, y):
      
      X_train, X_test = X_copy.iloc[train_index], X_copy.iloc[test_index]
      y_train, y_test = y.iloc[train_index], y.iloc[test_index]
      eval_dataset = lgb.Dataset(X_test, label=y_test)
      #
      #eval_set=eval_dataset,       Kept running into error complaining of no length, not clear why
      #early_stopping_rounds=200,
      model.fit(
        X_train,
        y_train,
        eval_metric="AUC",
        verbose=False)
      
      y_eval_predproba = model.predict_proba(X_test)
      new_auc_score = roc_auc_score(y_test, y_eval_predproba[:,1])
      print(f'New AUC Score {new_auc_score}')
      auc.append(float(new_auc_score))
      if auc_score < new_auc_score:
        auc_score = new_auc_score 
        keep_model = model
    
    ##########################
    # After KFold processing
    # Record the results to a file for further analysis outside the notebook
    ##########################
    print(f'all scores {auc}')
    print(f'Best Roc AUC score {auc_score}')
    result = {}
    result['optuna_score'] = float(auc_score). # score that is returned for the optuna trial
    
      
    result['scores'] = auc
    result['params'] = dict(keep_model.get_params()) 
    print(keep_model.get_params())

    # right now, write as individual dictionaries
    # This can be altered to make it more easily loadable into a pandas dataframe
    with open('./artifacts/lgbm_results.json', 'a') as f:
        json.dump(dict(result), f)
        f.write('\n')

    filename = f'./models/lgbm_{trial.number}'
    
    pickle.dump(keep_model, open(filename, 'wb'))
    
    return auc_score
    
def kfold_callback(study, trial):
    model_type = 'lgbm'
    # The callback is used to capture the best model.  It checks for the best trial number and renames the model for that trial
    # Persisting the best model, means that we can unpickle it and run further tests whle Optuna keeps iterating to find a better tuning
    filename = f'./models/{model_type}_{trial.number}'
    best_model_name = f'./models/best_{model_type}'
    
    if study.best_trial.number == trial.number:
        os.remove(best_model_name)
        os.rename(filename, best_model_name)
    else:
        os.remove(filename)
    print(f'filename {filename} best_model_name{best_model_name}  best_trial_number {study.best_trial.number}')

In [ ]:
# Create the best model file if it doesn't exist, otherwise don't alter it
# This file gets updated with the best scoring model of the study
best_model_name = './models/best_lgbm'
best_model_file = Path(best_model_name)
best_model_file.touch(exist_ok=True)

lgbm_study = optuna.create_study(direction='maximize')
lgbm_study.optimize(kfold_objective, n_trials=100, callbacks=[kfold_callback])
# Lightgbm has its own Optuna integration where it writes the best file, but
# since Catboost model persistence is still in the works, I just use a similar 
# callback for both

[I 2021-10-03 07:14:37,686] A new study created in memory with name: no-name-9c25405c-f871-4f69-8cb8-428ef60bcd02


STARTING Params: {'metric': 'AUC', 'verbosity': 0, 'device': 'gpu', 'boosting_type': 'gbdt', 'subsample': 0.5958302071958159, 'learning_rate': 0.008223238100425778, 'n_estimators': 2903, 'subsample_freq': 1, 'reg_alpha': 13, 'reg_lambda': 15, 'max_depth': 6, 'num_leaves': 4, 'feature_fraction': 0.9908605149315594, 'min_child_samples': 83, 'objective': 'binary'}
Param: {'metric': 'AUC', 'verbosity': 0, 'device': 'gpu', 'boosting_type': 'gbdt', 'subsample': 0.5958302071958159, 'learning_rate': 0.008223238100425778, 'n_estimators': 2903, 'subsample_freq': 1, 'reg_alpha': 13, 'reg_lambda': 15, 'max_depth': 6, 'num_leaves': 4, 'feature_fraction': 0.9908605149315594, 'min_child_samples': 83, 'objective': 'binary'}


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[LightGBM] [Warning] feature_fraction is set=0.9908605149315594, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9908605149315594
[LightGBM] [Warning] feature_fraction is set=0.9908605149315594, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9908605149315594
New AUC Score 0.8502842462324472


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



...

[LightGBM] [Warning] feature_fraction is set=0.7403721282077524, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7403721282077524
[LightGBM] [Warning] feature_fraction is set=0.7403721282077524, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7403721282077524
New AUC Score 0.8562027403011585


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[LightGBM] [Warning] feature_fraction is set=0.7403721282077524, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7403721282077524
[LightGBM] [Warning] feature_fraction is set=0.7403721282077524, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7403721282077524
New AUC Score 0.8574910263133066
all scores [0.8577228035313857, 0.8563604840495794, 0.8570609445086427, 0.8562027403011585, 0.8574910263133066]
Roc AUC score 0.8577228035313857
{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.013912800254196058, 'max_depth': 6, 'min_child_samples': 67, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 5655, 'n_jobs': -1, 'num_leaves': 10, 'objective': 'binary', 'random_state': 42, 'reg_alpha': 18, 'reg_lambda': 16, 'silent': True, 'subsample': 0.5769168603086565, 'subsample_for_bin': 200000, 'subsample_freq': 1, 'metric': 'AUC', 'verbosity': 0, 'device': 'gpu', 'f

[I 2021-10-04 04:08:44,265] Trial 99 finished with value: 0.8577228035313857 and parameters: {'boosting_type': 'gbdt', 'subsample': 0.5769168603086565, 'eta': 0.013912800254196058, 'n_estimators': 5655, 'reg_alpha': 18, 'reg_lambda': 16, 'max_depth': 6, 'num_leaves': 10, 'feature_fraction': 0.7403721282077524, 'min_child_samples': 67}. Best is trial 75 with value: 0.8592430823618162.


filename ./models/lgbm_99 best_model_name./models/best_lgbm  best_trial_number 75


In [1]:
 # To get the dictionary of parameter name and parameter values:
 # Right now logging is not working as expected so this is pending
#logging.info(f'Return a dictionary of parameter name and parameter values:{lgbm_study.best_params}')

results = []
print(f'Return a dictionary of parameter name and parameter values:{lgbm_study.best_params}')
#results.append({'msg':lgbm_study.best_params})
 
# To get the best observed value of the objective function:
#logging.info(f'Return the best observed value of the objective function: {lgbm_study.best_value}')
print(f'Return the best observed value of the objective function: {lgbm_study.best_value}')
#results.append({'msg':lgbm_study.best_value})
 
# To get the best trial:
print(f'Return the best trial: {lgbm_study.best_trial}')
#results.append({'msg': lgbm_study.best_trial})

pd.DataFrame(results).to_csv('results.csv', index=False)
 
# To get all trials:
#print("Return all the trials:", study.trials)

NameError: ignored

In [10]:
model_file = open('./models/best_lgbm', 'rb')
colab_lgbm_model_eval = pickle.load(model_file)
model_file.close()

In [14]:
colab_lgbm_model_eval.fit(X,y)
y_predproba = colab_lgbm_model_eval.predict_proba(X)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[LightGBM] [Warning] feature_fraction is set=0.7731920572320682, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7731920572320682
[LightGBM] [Warning] feature_fraction is set=0.7731920572320682, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7731920572320682


In [15]:
roc_auc_score(y, y_predproba[:, 1])

0.8661878759561727

Kaggle score

In [17]:
y_predproba = pd.DataFrame(colab_lgbm_model_eval.predict_proba(test_X))
y_predproba['id'] = y_predproba.index + len(X)
y_predproba.to_csv('./artifacts/lightgbm_predproba.csv', index=False)
results_df = y_predproba.rename(columns={1:'target'}).copy()
results_df = results_df[['id','target']]
results_df.to_csv('./artifacts/lightgbm_submission.csv', index=False)